In [ ]:
# pip install libraries needed
%%capture
!pip install lightweight_mmm 
!pip install numpy==1.20.3

In [ ]:
# define function needed for calculating the moving average
def convert_to_ma(df_series, column, window): 
    df_series['MA'] = df_series[column].rolling(window).mean()
    for i in range(window-1):
      df_series['MA'].loc[i] = df_series['MA'].loc[window-1]
    return df_series["MA"]

In [ ]:
# imorting all libraries
from google.colab import drive
import pandas as pd
import jax.numpy as jnp
import jax as jax
import numpyro
import numpy as np
import matplotlib.pyplot as plt
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils
from lightweight_mmm import media_transforms
from lightweight_mmm import models
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning 
import warnings

#supressor deprecation warning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

# mount drive to get acces to data files
drive.mount('/content/drive')

#set seed
SEED = 230129

In [ ]:
# import all data and convert to numpy array
drive_path = "drive/MyDrive/Colab Notebooks/Data/"

# define marketing spending per channel
df_marketing = pd.read_csv(drive_path + '230307_DATASET_MMM_v2.csv')
channels = ['S_SEM_BRAND_AON_LOWER','S_SEM_NON_BRAND_AON_LOWER','S_FB_DISPLAY_MKT_UPPER','S_FB_DISPLAY_PERF_LOWER','S_FB_DISPLAY_CORP_BRANDING','S_GOOGLE_DISPLAY_MKT_UPPER','S_GOOGLE_DISPLAY_PERF_LOWER','S_GOOGLE_DISPLAY_CORP_BRANDING','S_DISPLAY_METASEARCH_PERF_AON_LOWER','S_DV360_DISPLAY_MKT_UPPER','S_DV360_DISPLAY_CORP_BRANDING','S_RTBHOUSE_DISPLAY_MKT_UPPER','S_RTBHOUSE_DISPLAY_PERF_LOWER','S_CRITEO_DISPLAY_PERF_LOWER','S_METASEARCH_CORE_AON_LOWER','S_TV_BRANDING','S_OOH_BRANDING','S_RADIO_BRANDING','S_OTHER']
media_data = df_marketing[channels].to_numpy()

# define target, which is the revenue 
target = df_marketing["REVENUE"].to_numpy()

# define summation of cost per channel over full period
costs = np.sum(media_data, axis=0)

# define all external features
df_ts_economical = pd.read_csv(drive_path + '230307_chile_economical_ts.csv')
df_ts_airline = pd.read_csv(drive_path + '230307_airline_data.csv')
extra_features=pd.concat([df_ts_economical[['unemployment_rate_month','cpi_rate','price_clpusd']],
                          df_ts_airline[['n_purchases','avg_paid_fare_usd']],convert_to_ma(df_ts_airline, ['ASK_dom_C'], 20)], axis=1).to_numpy()


In [ ]:
# split the data into train and test set
split_point = int(len(df_marketing) * 0.8)

media_data_train = media_data[:split_point, ...]
media_data_test = media_data[split_point:, ...]
target_train = target[:split_point]
target_test = target[split_point:]
extra_features_train = extra_features[:split_point, ...]
extra_features_test = extra_features[split_point:, ...]

In [ ]:
# define scaling factors for each data set
media_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by=0.15)
extra_features_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)

# scale all data such that all variables are in the same range
media_data_train = media_scaler.fit_transform(media_data_train)
media_data_test = media_scaler.fit_transform(media_data_test)
target_train = target_scaler.fit_transform(target_train)
target_test = target_scaler.fit_transform(target_test)
costs = cost_scaler.fit_transform(costs)
extra_features_train = extra_features_scaler.fit_transform(extra_features_train)
extra_features_test = extra_features_scaler.fit_transform(extra_features_test)

In [ ]:
# make LightweightMMM object
mmm = lightweight_mmm.LightweightMMM(model_name="adstock")

# fit model to the data
mmm.fit(
      media                 =media_data_train,
      target                =target_train,
      extra_features        =extra_features_train,
      media_prior           =costs,
      number_warmup         =2000,
      number_samples        =2000,
      degrees_seasonality   =2,
      seasonality_frequency =365,
      weekday_seasonality   =False,
      target_accept_prob    =0.7,
      seed=SEED)      

In [ ]:
# show model paramter estimates
print(channels)
mmm.print_summary()

In [ ]:
# show in sample model fit
plot.plot_model_fit(mmm, target_scaler=target_scaler)

In [ ]:
# show out-of-sample forcasting
new_predictions = mmm.predict(media=media_data_test,target_scaler=target_scaler)

if len(new_predictions[0]) == len(target_test):
  N=len(new_predictions[0])
  sum=0
  for i in range(N):
    sum += abs(target_test[i]-new_predictions[0][i])/target_test[i]
  MAPE = 100* sum / N

print(f"MAPE: {MAPE}")
plot.plot_out_of_sample_model_fit(out_of_sample_predictions=new_predictions, out_of_sample_target=target_test)

In [ ]:
# Do budget forecasting
n_media_channels = len(channels)
prices = jnp.ones(mmm.n_media_channels)
n_time_periods = int(data_size*0.15)
budget = jnp.sum(jnp.dot(prices, media_data.mean(axis=0)))* n_time_periods

# Run optimization with the parameters of choice.
solution, kpi_without_optim, previous_media_allocation = optimize_media.find_optimal_budgets(
    media_mix_model=mmm,
    budget=budget,
    prices=prices,
    media_scaler=media_scaler,
    target_scaler=target_scaler,
    extra_features=extra_features_scaler.transform(extra_features_test)[:n_time_periods],
    n_time_periods=n_time_periods,
    seed=SEED)

# Obtain the optimal allocation.
optimal_buget_allocation = prices * solution.x
print(optimal_buget_allocation)

# similar renormalization to get previous budget allocation
previous_budget_allocation = prices * previous_media_allocation
print(previous_budget_allocation)

# Both numbers should be almost equal
print(budget), print(jnp.sum(solution.x * prices))

# Plot out pre post optimization budget allocation and predicted target variable comparison.
plot.plot_pre_post_budget_allocation_comparison(media_mix_model=mmm, 
                                                kpi_with_optim=solution['fun'], 
                                                kpi_without_optim=kpi_without_optim,
                                                optimal_buget_allocation=optimal_buget_allocation, 
                                                previous_budget_allocation=previous_budget_allocation, 
                                                figure_size=(20,10))